In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as skd
import sklearn.preprocessing as skp
import sklearn.metrics as skmt
rng = np.random.default_rng(100)
scaler = skp.StandardScaler()

In [ ]:
def add_noise(X, label, p):
    n, d = X.shape
    r = int(p * n)
    noise = rng.uniform(0, 1, (r, d))
    noise = X.min(0) + (X.max(0) - X.min(0)) * noise
    noise_index = np.zeros(r)
    noise_index.fill(-1)
    
    new_X = np.concatenate((X, noise))
    new_label = np.concatenate((label, noise_index))
    return new_X, new_label

In [ ]:
def calculate_U(X, V):
    n, d = X.shape
    U = np.zeros(n)
    for i in range(n):
        dist = ((X[i] - V) * (X[i] - V)).sum(1)
        j = dist.argmin()
        U[i] = j
    return U

In [ ]:
def calculate_V(X, U, k):
    n, d = X.shape
    V = np.zeros([k, d])
    for j in range(k):
        index = np.where(U == j)[0]
        r = index.shape[0]
        if r == 0:
            V[j] = rng.choice(X)
        else:
            for i in index:
                V[j] = V[j] + X[i]
            V[j] = V[j] / r
    return V

In [ ]:
def k_means(X, k):
    n, d = X.shape
    U = np.zeros(n)
    V = rng.choice(X, k)
    
    condition = True
    while(condition):
        temp = np.copy(U)
        U = calculate_U(X, V)
        V = calculate_V(X, U, k)
        condition = not np.array_equal(U, temp)

    return U

In [ ]:
def calculate_W(X, k_max):
    W = np.zeros(k_max)

    for k in range(k_max):
        valid = False
        while not valid:
            Z = k_means(X, k + 1)
            valid = True
            for r in range(k + 1):
                I = np.where(Z == r)[0]
                nr = I.shape[0]
                if nr == 0:
                    valid = False
                    break
        for r in range(k + 1):
            I = np.where(Z == r)[0]
            nr = I.shape[0]
            Dr = 0
            for i in range(nr):
                for j in range(i + 1, nr):
                    Dr = Dr + ((X[I[i]] - X[I[j]]) * (X[I[i]] - X[I[j]])).sum()
            W[k] = W[k] + (Dr / (2 * nr))

    return W

In [ ]:
def calculate_optimal(X, k_max):
    n, d = X.shape
    W = calculate_W(X, k_max)

    W_B = np.zeros([B_, k_max])
    for b in range(B_):
        X_b = rng.uniform(0, 1, (n, d))
        X_b = X.min(0) + (X.max(0) - X.min(0)) * X_b
        W_B[b] = calculate_W(X_b, k_max)

    k_opt = 1

    gap = (np.log(W_B[:, 0])).mean() - np.log(W[0])
    sd = ((np.log(W_B[:, 0]) - (np.log(W_B[:, 0])).mean()) ** 2).mean()
    s = sd * np.sqrt(1 + (1 / B_))
    for k in range(1, k_max):
        prev_gap = gap
        prev_sd = sd
        prev_s = s
        gap = (np.log(W_B[:, k])).mean() - np.log(W[k])
        sd = ((np.log(W_B[:, k]) - (np.log(W_B[:, k])).mean()) ** 2).mean()
        s = sd * np.sqrt(1 + (1 / B_))
        if prev_gap > gap - s:
            k_opt = k
            break

    return k_opt

In [ ]:
def d_phi(x, y):
    return (x * np.log(x / y)).sum() - (x - y).sum()

In [ ]:
def calculate_I(X, q, V, k):
    n, d = X.shape
    dist = np.zeros(n)
    for i in range(n):
        temp = np.zeros(k)
        for j in range(k):
            temp = d_phi(X[i], V[j])
        dist[i] = temp.min()
    I = np.argpartition(dist, q)[: q]
    return I

In [ ]:
def calculate_Z(X, I, V, k):
    n, d = X.shape
    Z = np.zeros(n)
    Z.fill(k)
    for i in I:
        dist = np.zeros(k)
        for j in range(k):
            dist = d_phi(X[i], V[j])
        j = dist.argmin()
        Z[i] = j
    return Z

In [ ]:
# data, label = skd.make_blobs(n_samples = 500, random_state = 100)
data, label = skd.make_circles(n_samples = 250, factor = 0.25, noise = 0.05, random_state = 0)

t = np.arange(250)
q = 250 - 1

for m in range(5):
    
    print("Noise :", 0.05 * m)
    print("Iterations :", end = " ")

    k_star = []
    ari = []
    nnmi = []

    for h in range(15):

        X, label = add_noise(data, label, 0.05 * m)
        X = scaler.fit_transform(X)
        X = X - X.min(0) + 1
    
        n, d = X.shape
        k_max = 2 * np.unique(label).shape[0]
        B_ = 25
        k_opt = calculate_optimal(X, k_max)
    
        Z = np.zeros(n)
        V = rng.choice(X, k_opt)

        condition = True
        while(condition):
            temp = np.copy(Z)
            I = calculate_I(X, q, V, k_opt)
            Z = calculate_Z(X, I, V, k_opt)
            V = calculate_V(X, Z, k_opt)
            condition = not np.array_equal(Z, temp)

        if (m == 2) and (h == 1):

            plt.figure()
            plt.scatter(X[:, 0], X[:, 1], c = label, alpha = 0.5)
            plt.grid()
            plt.savefig("robust_bregman_original.png")
    
            plt.figure()
            plt.scatter(X[:, 0], X[:, 1], c = Z, alpha = 0.5)
            plt.grid()
            plt.savefig("robust_bregman.png")

        label = label[t]
        Z = Z[t]

        print((h + 1), end = " ")
        k_star.append(np.unique(Z).shape[0])
        ari.append(skmt.adjusted_rand_score(label, Z))
        nnmi.append(skmt.adjusted_mutual_info_score(label, Z))

    print()
    print("Number of clusters :", np.array(k_star).mean(), "+/-", np.array(k_star).std())
    print("ARI :", np.array(ari).mean(), "+/-", np.array(ari).std())
    print("NNMI :", np.array(nnmi).mean(), "+/-", np.array(nnmi).std())
    print()